### Entrenamiento del modelo de YOLOv8 con el dataset de imagenes etiquetadas

#### Instalación de la libreria ultralutics proveedora de YOLO


In [72]:
!pip install ultralytics

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

#### Importacion de otras librerias necesarias para el proceso

#### Conexión con la unidad de drive donde estan las imagenes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Definición de las rutas donde se van a alojar las imagenes dentro del entorno del colab para el entrenamiento

In [ ]:
train_path_img = "./yolo_data/images/train"
train_path_label = "./yolo_data/labels/train"
val_path_img = "./yolo_data/images/val"
val_path_label = "./yolo_data/labels/val"
test_path = "./yolo_data/test"

#### División del set de datos en entrenamiento y prueba

In [ ]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.png',f"{train_path_img}/" + filex + '.png' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".png", f"{train_path_img}/" + filex + '.png')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.png', f"{val_path_img}/" + filex + '.png' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
# train_test_split('/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/data/') ### without negative images
#train_test_split('/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/data/','/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/negativas/') ### if you want to feed negative images
train_test_split('/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/data/','/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/negativas/')


------ PROCESS STARTED -------
--- This folder has a total number of 123 images---


  0%|          | 0/99 [00:00<?, ?it/s]

------ Training data created with 80% split 99 images -------


  0%|          | 0/9 [00:00<?, ?it/s]

------ Total  9 negative images added to the training data -------
------ TOTAL Training data created with 108 images -------


  0%|          | 0/24 [00:00<?, ?it/s]

------ Testing data created with a total of 24 images ----------
------ TASK COMPLETED -------


Importación de la libreria ultralytics y verificación de su funcionamiento y los recursos que utilizará

In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.8/112.6 GB disk)


#### Entrenamiento del modelo en modo deteccion con el modelo preentrenado yolov8n.pt 300 épocas y tamaño de lote 1

In [ ]:
#!yolo task=detect mode=train model=yolov8n.pt data=/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/dataset.yaml epochs=300 imgsz=640 batch=6 project=/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/training_results name=controlPastaTermica

#entreno con 10 epocas
!yolo task=detect mode=train model=yolov8n.pt data=/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/dataset.yaml epochs=20 imgsz=640 batch=10 project=/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/training_results name=controlPastaTermica
train_test_split('/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/data/','/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/negativas/')



2. Generación de métricas de evaluación
Después del entrenamiento, evalúa el modelo en el conjunto de prueba:

Explicación de las métricas:

1. **`metrics.box.p` (Precisión para cada clase):**  
   La precisión mide qué tan bien el modelo identifica correctamente los objetos de cada clase.  
   Por ejemplo, si el modelo detecta 10 objetos y 9 son correctos, la precisión será alta (90%). Una precisión alta significa que hay pocos "falsos positivos" (detecciones incorrectas).

2. **`metrics.box.r` (Recall para cada clase):**  
   El recall mide qué tan bien el modelo encuentra todos los objetos reales de cada clase.  
   Por ejemplo, si hay 10 objetos reales y el modelo detecta 8, el recall será del 80%. Un recall alto significa que hay pocos "falsos negativos" (objetos que el modelo no detectó).

3. **`metrics.box.map50` (mAP a IoU de 0.5):**  
   El *Mean Average Precision* (mAP) combina precisión y recall, pero en este caso se calcula cuando la superposición entre la predicción y el objeto real (IoU) es al menos 50%. Es un buen indicador general de qué tan bien funciona el modelo.

4. **`metrics.box.map` (mAP promedio en IoU de 0.5 a 0.95):**  
   Este mAP evalúa el desempeño del modelo considerando diferentes niveles de superposición, desde un 50% hasta un 95%. Es un criterio más estricto, ya que verifica si las predicciones son muy precisas.

### Resultado Esperado:  
El código calculará estos valores y mostrará el promedio de:
- Precisión: Qué tan exactas son las detecciones.
- Recall: Qué tan completo es el modelo detectando objetos.
- mAP@0.5: Qué tan bien funciona el modelo con una tolerancia estándar.
- mAP@0.5:0.95: Qué tan bien funciona el modelo con tolerancias más estrictas.  

Este análisis ayuda a entender si el modelo está funcionando correctamente o si necesita ajustes.

In [ ]:
from ultralytics import YOLO

# Configurar rutas
trained_model_path = '/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results/controlPastaTermica/weights/best.pt'

# Cargar el modelo entrenado
model = YOLO(trained_model_path)

# Evaluar el modelo en el conjunto de validación
metrics = model.val(data='/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/dataset.yaml')

# Acceder a las métricas desde el atributo 'box'
precision = metrics.box.p  # Precisión por clase
recall = metrics.box.r     # Recall por clase
map50 = metrics.box.map50  # mAP@0.5
map = metrics.box.map      # mAP@0.5:0.95

# Mostrar métricas principales
print("---- Métricas de evaluación ----")
print(f"Precisión promedio: {sum(precision) / len(precision):.4f}")
print(f"Recall promedio: {sum(recall) / len(recall):.4f}")
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map:.4f}")


Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/yolo_data/labels/val.cache... 24 images, 0 backgrounds, 0 corrupt: 100%|██████████| 24/24 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.04s/it]


                   all         24        167      0.996      0.995      0.995      0.919
                    OK         10         60      0.994          1      0.995      0.894
                    SP         10         47      0.995          1      0.995      0.934
                    NC          9         60          1      0.985      0.995      0.928
Speed: 7.3ms preprocess, 4.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val5
---- Métricas de evaluación ----
Precisión promedio: 0.9962
Recall promedio: 0.9951
mAP@0.5: 0.9950
mAP@0.5:0.95: 0.9186


#### Carga del modelo y exportacion en formato onnx (optimizado para uso en CPU)

In [ ]:
from ultralytics import YOLO
#model = YOLO("/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/training_results/controlPastaTermica8/weights/best.pt")  # load a custom trained model

model = YOLO("/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results/controlPastaTermica/weights/best.pt")
# Exportar el modelo
model.export(format="onnx")

Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results/controlPastaTermica/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (6.0 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.39...
ONNX: export success ✅ 1.6s, saved as '/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results/controlPastaTermica/weights/best.onnx' (11.7 MB)

Export complete (2.2s)
Results saved to /content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results/controlPastaTermica/weights
Predict:         yolo predict task=detect model=/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results/controlPastaTermica/weights/b

'/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results/controlPastaTermica/weights/best.onnx'

código para Google Colab que calcula métricas clave (precisión, recall, F1-score) para evaluar un modelo YOLOv8 utilizando un conjunto de datos dividido en entrenamiento y prueba por el código proporcionado. Este script asume que ya tienes un modelo YOLO entrenado y se usa para evaluar su rendimiento en el conjunto de prueba.

Código para Colab

#### Deteccion en los videos de prueba

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/training_results/controlPastaTermica8/weights/best.pt conf=0.77 source=/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/videos


Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 940, in entrypoint
    model = YOLO(model, task=task)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/models/yolo/model.py", line 23, in __init__
    super().__init__(model=model, task=task, verbose=verbose)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 145, in __init__
    self._load(model, task=task)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 285, in _load
    self.model, self.ckpt = attempt_load_one_weight(weights)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py", line 910, in attempt_load_one_weight
    ckpt, weight = torch_safe_load(weight)  # load ckpt
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py", line 837, in torch_safe_load
    ckpt = torch.load(fi

#### Copia de los videos con las predicciones realizadas desde el entorno de colab a una carpeta de Drive

In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/output